In [27]:
import sys
model_dir='/home/draif/Recipes/modelzoo'
sys.path.insert(0,model_dir)
import vgg16
import lasagne
import theano

In [22]:
from tqdm import tqdm
from skimage import transform, io
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Константы

In [11]:
DATA_FILE = "/home/draif/data/fer2013.csv"
INPUT_SHAPE = (224, 224)

## Загружаем данные

In [19]:
from collections import defaultdict

In [16]:
def prepare_image(image):
    return transform.resize(
        np.array(map(lambda x: float(x) / 255, image.split())).reshape((48,48)),
        INPUT_SHAPE
    )

In [20]:
DATA = defaultdict(list)
GROUND_TRUTH = defaultdict(list)

with open(DATA_FILE, 'r') as f:
    f.readline()
    for line in tqdm(f.readlines()):
        emotion, pixels, usage = line.strip().split(',')
        DATA[usage].append(prepare_image(pixels))
        GROUND_TRUTH[usage].append(emotion)

100%|██████████| 35887/35887 [01:57<00:00, 304.83it/s]


In [25]:
X_train = DATA['Training']
X_val = DATA['PublicTest']
Y_train = GROUND_TRUTH['Training']
Y_val = GROUND_TRUTH['PublicTest']

X_test = DATA['PrivateTest']
Y_test = GROUND_TRUTH['PrivateTest']

## Подготовка к обучению

In [26]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs), batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [28]:
net = vgg16.build_model()

MemoryError: ('Error allocating 67108864 bytes of device memory (out of memory).', "you might consider using 'theano.shared(..., borrow=True)'")